In [1]:
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
from PIL import Image
import torchvision.transforms as transforms
from IPython.display import display
import tensorrt as trt
import cv2

In [2]:
def apply_cm(
    image: np.array, map: np.array, threshold: float = 0.5, alpha: float = 0.5
) -> np.array:
    """Apply color map on image

    Args:
        image (np.array): Image
        map (np.array): Grayscale, unnormalized cm.
        threshold (float, optional): Threshold for map. Defaults to 0.5.
        alpha (float, optional): Transparency. Defaults to 0.5.

    Returns:
        np.array: Image with applied cm.
    """
    map_thresholded = np.where(map > threshold, 1, 0)
    map_thresholded = (map_thresholded * 255).astype(np.uint8)
    map_thresholded = cv2.cvtColor(map_thresholded, cv2.COLOR_GRAY2RGB)

    map_normalized = cv2.normalize(
        map, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U
    )
    map_colored = cv2.applyColorMap(map_normalized, cv2.COLORMAP_JET)
    blended_image = cv2.addWeighted(image, 1 - alpha, map_colored, alpha, 0)

    return blended_image

In [3]:
def display_img_with_map(out: np.array, map: np.array, image: np.array) -> None:
    """Displays image with explainability map.

    Args:
        out (np.array): Neural network prediction.
        map (np.array): Neural network explainability output.
        image (np.array): Original inference image.
    """
    preds = out > 0.5
    for i, pred in enumerate(preds):
        map_segmentation = map[i] if pred == 1 else np.zeros_like(map[i])
        if pred == 1:
            map_segmentation = (map_segmentation - map_segmentation.min()) / (
                map_segmentation.max() - map_segmentation.min()
            )
        blended_img = apply_cm(image, map_segmentation)

        # Only for displaying
        blended_img = cv2.cvtColor(blended_img, cv2.COLOR_BGR2RGB)
        display(Image.fromarray(blended_img))

In [4]:
# https://stackoverflow.com/questions/59280745/inference-with-tensorrt-engine-file-on-python
class TensorRTInference:
    def __init__(self, engine_path):
        # initialize
        self.logger = trt.Logger(trt.Logger.ERROR)
        self.runtime = trt.Runtime(self.logger)

        # setup
        self.engine = self.load_engine(engine_path)
        self.context = self.engine.create_execution_context()

        # allocate buffers
        self.inputs, self.outputs, self.bindings, self.stream = self.allocate_buffers(
            self.engine
        )

    def load_engine(self, engine_path):
        # loads the model from given filepath
        with open(engine_path, "rb") as f:
            engine = self.runtime.deserialize_cuda_engine(f.read())
        return engine

    class HostDeviceMem:
        def __init__(self, host_mem, device_mem, shape):
            # keeping track of addresses
            self.host = host_mem
            self.device = device_mem
            # keeping track of shape to un-flatten it later
            self.shape = shape

    def allocate_buffers(self, engine):
        inputs, outputs, bindings = [], [], []
        stream = cuda.Stream()

        for i in range(engine.num_io_tensors):
            tensor_name = engine.get_tensor_name(i)
            shape = engine.get_tensor_shape(tensor_name)
            size = trt.volume(shape)
            dtype = trt.nptype(engine.get_tensor_dtype(tensor_name))

            # allocate host and device buffers
            host_mem = cuda.pagelocked_empty(size, dtype)
            device_mem = cuda.mem_alloc(host_mem.nbytes)

            # append the device buffer address to device bindings
            bindings.append(int(device_mem))

            # append to the appropiate input/output list
            if engine.get_tensor_mode(tensor_name) == trt.TensorIOMode.INPUT:
                inputs.append(self.HostDeviceMem(host_mem, device_mem, shape))
            else:
                outputs.append(self.HostDeviceMem(host_mem, device_mem, shape))

        return inputs, outputs, bindings, stream

    def infer(self, input_data):
        # transfer input data to device
        np.copyto(self.inputs[0].host, input_data.ravel())
        cuda.memcpy_htod_async(self.inputs[0].device, self.inputs[0].host, self.stream)

        # set tensor address
        for i in range(self.engine.num_io_tensors):
            self.context.set_tensor_address(
                self.engine.get_tensor_name(i), self.bindings[i]
            )

        # run inference
        self.context.execute_async_v3(stream_handle=self.stream.handle)

        # transfer predictions back
        for i in range(len(self.outputs)):
            cuda.memcpy_dtoh_async(
                self.outputs[i].host, self.outputs[i].device, self.stream
            )

        # synchronize the stream
        self.stream.synchronize()

        # un-flatten the outputs
        outputs = []
        for i in range(len(self.outputs)):
            output = self.outputs[i].host
            output = output.reshape(self.outputs[i].shape)
            outputs.append(output)

        return outputs

In [5]:
engine_path = 'efficientnet_v2_s_downscaled_pcb_fp32.trt'
trt_inference = TensorRTInference(engine_path)

In [6]:
image = cv2.imread('../docs/sample_data/01_short_04_1926_1070.png', cv2.IMREAD_COLOR)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
transform = transforms.Compose([transforms.ToTensor()])
tensor = transform(image).unsqueeze(0)

In [ ]:
%%timeit
trt_inference.infer(tensor.numpy())

In [ ]:
outputs = trt_inference.infer(tensor.numpy())
display_img_with_map(outputs[0], outputs[1], image)

In [9]:
engine_path = 'efficientnet_v2_s_downscaled_pcb_fp16.trt'
trt_inference = TensorRTInference(engine_path)

In [ ]:
%%timeit
trt_inference.infer(tensor.numpy())

In [ ]:
outputs = trt_inference.infer(tensor.numpy())
display_img_with_map(outputs[0], outputs[1], image)

In [20]:
engine_path = 'vit_tiny_patch16_224.augreg_in21k_ft_in1k_pcb_fp32.trt'
trt_inference = TensorRTInference(engine_path)

In [ ]:
%%timeit
trt_inference.infer(tensor.numpy())

In [ ]:
outputs = trt_inference.infer(tensor.numpy())
display_img_with_map(outputs[0], outputs[1], image)

In [23]:
engine_path = 'vit_tiny_patch16_224.augreg_in21k_ft_in1k_pcb_fp16.trt'
trt_inference = TensorRTInference(engine_path)

In [ ]:
%%timeit
trt_inference.infer(tensor.numpy())

In [ ]:
outputs = trt_inference.infer(tensor.numpy())
display_img_with_map(outputs[0], outputs[1], image)